## Import Library

In [1]:
# !pip install  dgl -f https://data.dgl.ai/wheels/cu121/repo.html
# !pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html
# !pip install cudart
# !pip install pyspark
# !pip install py4j

In [1]:
import os, sys
os.environ['DGLBACKEND'] = 'pytorch'
from copy import deepcopy
import numpy as np
import pandas as pd
import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import networkx as nx
import itertools
from itertools import combinations
import matplotlib.pyplot as plt
import dgl
from dgl import function as fn
from dgl.dataloading import DataLoader

%matplotlib inline
os.environ['DGLBACKEND'] = 'pytorch'
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()

spark = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, Bucketizer, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

KeyboardInterrupt: 

## Load Dataset

In [ ]:
df_test = pd.read_csv('data/test.csv')
df_test['dataset'] = 'test'
df_test = pd.read_csv('data/training.csv')

In [ ]:
df_test.info()

In [ ]:
from kafka import KafkaProducer
import json
from pyspark.sql import SparkSession
from pyspark.sql import Row
from datetime import datetime

spark = SparkSession.builder.appName("KafkaStreaming").getOrCreate()

def convert_timestamp(ts):
    return ts.strftime("%Y-%m-%d %H:%M:%S")

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v, default=str).encode('utf-8')
)

# topic = 'test_streaming'
# for index, row in df_test.iterrows():
#     row_dict = row.to_dict()
#     row_dict = {key: convert_timestamp(value) if isinstance(value, datetime) else value for key, value in row_dict.items()}

#     producer.send(topic, value=row_dict)
#     print(f"Sent message: {row_dict}")
# producer.close()

topic = 'test_streaming'
batch_size = 5000
total_rows = len(df_test)

# Calculate the number of batches
num_batches = (total_rows + batch_size - 1) // batch_size

# Get the starting and ending indices for each batch
for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size if (batch_num + 1) * batch_size < total_rows else total_rows

    for index, row in df_test.iloc[start_idx:end_idx].iterrows():
        row_dict = row.to_dict()
        row_dict = {key: convert_timestamp(value) if isinstance(value, datetime) else value for key, value in row_dict.items()}

        producer.send(topic, value=row_dict)
        print(f"Sent message: {row_dict}")

    # Flush the producer after processing each batch
    producer.flush()